In [3]:
# Import necessary libraries
import ee
import geemap

# Authenticate to Earth Engine
#ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


In [4]:


# Define the FAO/GAUL administrative units dataset
dataset = ee.FeatureCollection('FAO/GAUL/2015/level2')

# Define a region of interest (ROI) for India
india_roi = ee.Geometry.Polygon(
    [[[
      68.11137880340998, 7.96553484576015],
      [88.3927306205192, 7.96553484576015],
      [88.3927306205192, 35.67459030295267],
      [68.11137880340998, 35.67459030295267],
      [68.11137880340998, 7.96553484576015]
    ]]
)


Map


NameError: name 'Map' is not defined

In [5]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the FAO/GAUL administrative units dataset
dataset = ee.FeatureCollection('FAO/GAUL/2015/level2')

# Define a region of interest (ROI) for India
india_roi = ee.Geometry.Polygon(
    [[[
      68.11137880340998, 7.96553484576015],
      [88.3927306205192, 7.96553484576015],
      [88.3927306205192, 35.67459030295267],
      [68.11137880340998, 35.67459030295267],
      [68.11137880340998, 7.96553484576015]
    ]]
)

# Filter the administrative units dataset to select India
india_dataset = dataset.filterBounds(india_roi)

# Define a region of interest (ROI) for New Delhi
delhi_roi = ee.Geometry.Point(77.216721, 28.613939).buffer(0.5)

# Filter the India dataset to select New Delhi
delhi_feature = india_dataset.filterBounds(delhi_roi)

# Print the selected administrative unit (New Delhi)
print("Selected Administrative Unit (New Delhi):")
selected_features = delhi_feature.getInfo()
for feature in selected_features['features']:
    print("Admin Unit ID:", feature['properties']['ADM2_CODE'])
    print("Admin Unit Name:", feature['properties']['ADM2_NAME'])

# Create a Map
Map = geemap.Map()

# Set the Map center
Map.setCenter(12.876, 42.682, 5)

# Add the administrative units dataset to the map
Map.centerObject(india_dataset, 5)
Map.addLayer(india_dataset, {'color': '00909F'}, 'Second Level Administrative Units')

# Display New Delhi
Map.centerObject(delhi_feature, 8)
Map.addLayer(delhi_feature, {'color': 'FF0000'}, 'New Delhi')

# Display the Map
Map


Selected Administrative Unit (New Delhi):
Admin Unit ID: 17633
Admin Unit Name: Delhi


Map(center=[28.646455931994893, 77.10900424302301], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
# Function to get Landsat imagery
def get_landsat_imagery(region, start_date, end_date):
    landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
        .filterBounds(region) \
        .filterDate(start_date, end_date) \
        .sort("CLOUD_COVER")
    return landsat

# Define the time ranges for Landsat imagery
years = list(range(2000, 2021, 4))
time_ranges = [(datetime.datetime(year, 1, 1), datetime.datetime(year, 12, 31)) for year in years]

# Get Landsat imagery for each time range
landsat_images = [get_landsat_imagery(delhi_feature.geometry(), start_date, end_date).first() for start_date, end_date in time_ranges]

# Calculate land cover for each Landsat image
landcover_images = [calculate_landcover(image) for image in landsat_images]

# Add the land cover layers to the map
for i, image in enumerate(landcover_images):
    Map.addLayer(image.select('Landcover'), {'min': 0, 'max': 255, 'palette': ['black', 'green', 'brown']}, f'Land Cover {years[i]}')


NameError: name 'datetime' is not defined

In [7]:
# Filter the administrative units dataset to select administrative 2 regions surrounding Delhi
delhi_surrounding_admin2 = india_dataset.filterBounds(delhi_roi).filter(ee.Filter.eq('ADM2_CODE', '2'))

# Clip the study area to the administrative units around Delhi
clipped_study_area = india_dataset.filterBounds(delhi_surrounding_admin2)

# Export the clipped study area as a vector shapefile
export_params = {
    'collection': clipped_study_area,
    'description': 'Study_Area',
    'fileFormat': 'SHP'  # Export as a shapefile
}

# Define the export task
export_task = ee.batch.Export.table.toDrive(**export_params)

# Start the export task
export_task.start()

# Wait for the export task to complete
import time
while export_task.status()['state'] == 'RUNNING':
    time.sleep(5)

# Check if the export task completed successfully
if export_task.status()['state'] == 'COMPLETED':
    # Get the URL of the exported file
    export_url = export_task.status()['destinationUrl']
    print("File saved at:", export_url)
else:
    print("Export task failed:", export_task.status())


Export task failed: {'state': 'READY', 'description': 'Study_Area', 'creation_timestamp_ms': 1707507609362, 'update_timestamp_ms': 1707507609362, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'KN5ZDGDQJXOVGJ3ZQ6P44FTQ', 'name': 'projects/earthengine-legacy/operations/KN5ZDGDQJXOVGJ3ZQ6P44FTQ'}


In [8]:
import datetime
# Create a Map
Map = geemap.Map()
Map.centerObject(india_roi, 5)  # Center the map on India

# Add a layer for New Delhi
Map.addLayer(delhi_feature, {'color': 'FF0000'}, 'New Delhi')

# Buffer New Delhi and filter surrounding administrative units
buffered_delhi = delhi_feature.geometry().buffer(0.1)  # Adjust the buffer distance as needed
surrounding_admin_units = india_dataset.filterBounds(buffered_delhi)

# Add a layer for surrounding administrative units
Map.addLayer(surrounding_admin_units, {'color': 'Yellow'}, 'Surrounding Administrative Units')

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Function to calculate land cover
def calculate_landcover(image):
    landcover = image.select('landcover').rename('Landcover')
    return image.addBands(landcover)

# Function to get Landsat imagery
def get_landsat_imagery(region, start_date, end_date):
    landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
        .filterBounds(region) \
        .filterDate(start_date, end_date) \
        .sort("CLOUD_COVER") \
        .first()
    return landsat

# Define the time range for Landsat imagery
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2021, 12, 31)

# Get Landsat imagery for New Delhi
landsat_delhi = get_landsat_imagery(delhi_feature.geometry(), start_date, end_date)

# Get Landsat imagery for surrounding administrative units
landsat_surrounding = get_landsat_imagery(buffered_delhi, start_date, end_date)

# Calculate NDVI for New Delhi
delhi_with_ndvi = calculate_ndvi(landsat_delhi)

# Calculate NDVI for surrounding administrative units
surrounding_admin_with_ndvi = calculate_ndvi(landsat_surrounding)

# Combine the NDVI images for Delhi and surrounding administrative units
combined_ndvi = delhi_with_ndvi.addBands(surrounding_admin_with_ndvi)

# Calculate land cover for New Delhi
delhi_with_landcover = calculate_landcover(landsat_delhi)

# Calculate land cover for surrounding administrative units
surrounding_admin_with_landcover = calculate_landcover(landsat_surrounding)

# Combine the land cover images for Delhi and surrounding administrative units
combined_landcover = delhi_with_landcover.addBands(surrounding_admin_with_landcover)

# Add the combined NDVI and land cover layers to the map
Map.addLayer(combined_ndvi.select('NDVI'), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI Combined')
Map.addLayer(combined_landcover.select('Landcover'), {'min': 0, 'max': 255, 'palette': ['black', 'green', 'brown']}, 'Land Cover Combined')

# Add a layer control to the map to toggle between layers
Map.addLayerControl()

# Display the Map
Map


EEException: Image.select: Pattern 'landcover' did not match any bands.

In [28]:
# Start the export task
export_task.start()

# Wait for the export task to complete
import time
while export_task.status()['state'] == 'RUNNING':
    time.sleep(30)  # Increased delay to 30 seconds
    print("Export task is still running...")

# Check if the export task completed successfully
if export_task.status()['state'] == 'COMPLETED':
    # Get the URL of the exported file
    export_url = export_task.status()['destinationUrl']
    print("File saved at:", export_url)
else:
    print("Export task failed:", export_task.status())


EEException: A different Operation was already started with the given request_id.